In [1]:
import time
import numpy as np
import pandas as pd
import seaborn as sb
from IPython.display import display, clear_output 
from sklearn.metrics import classification_report, mean_squared_error
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 9)

# set random seed to achieve same results each time
np.random.seed(23)

In [11]:
class FacialKeypoints():
    
    def __init__(self):
        self.train_csv = pd.read_csv("training.csv")
        self.test_csv = pd.read_csv("test.csv")
        self.sample_csv = pd.read_csv("SampleSubmission.csv")
        self.ID_csv = pd.read_csv("IdLookupTable.csv")
        self.label_names = self.train_csv.columns.tolist()[:-1]
        self.create_training(self.train_csv)
    
    def create_training(self, train_csv, dev_size=1000, nans=True):
        if nans == False:
            train_csv = train_csv.dropna()
        labels = train_csv.loc[:, train_csv.columns != 'Image'].values
        data = train_csv['Image'].str.split()
        data = np.vstack(data.apply(lambda row: np.asarray([int(n) for n in row])).values)
        data = data/255.0                                         # Rescale grayscale values to [0,1].
        shuffle = np.random.permutation(np.arange(data.shape[0])) # Shuffle the data
        data, labels = data[shuffle], labels[shuffle]             # Splitting into dev and training
        self.dev_data,   self.dev_labels   = data[:dev_size], labels[:dev_size]
        self.train_data, self.train_labels = data[dev_size:], labels[dev_size:]
        return 
    
    def plot_example(self, data, label):
        plt.imshow(data.reshape(96, 96), cmap='gray')
        plt.scatter(label[0::2], label[1::2], c='red', marker='x')
        plt.axis('off')
        return
    
    def _score(y_pred, y_actual):
        return
    
    def preprocessor():
        return
    
    def generate_training():
        return
    
    def blur_training():
        return
    
    def K_nn(k):
        return
    
    def LogReg(alpha):
        return
    
    def MLPRegressor():
        return
    
    pass

In [12]:
self = FacialKeypoints()

In [ ]:
self.plot_example(self.train_data[25], self.train_labels[25])

In [13]:
self.create_training(self.train_csv, nans=False)